In [2]:
import torch
import torchvision

!pip install --upgrade torchvision
!pip install cython
!pip install -U "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-w8zn1d9w
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-w8zn1d9w
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263910 sha256=b884632e6e2fc74934725824abefc7df5b12074d1f37d0796780ed22fbc52f02
  Stored in directory: /tmp/pip-ephem-wheel-cache-g78xa455/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


In [ ]:
# !cd Desktop/FSDL-FinalProject/dataverse_files/HAM10000_images_part_1/
# !ls
# from PIL import Image
# Image.open('ISIC_0024306.jpg')

In [15]:
# precision = 'fp32'
# pretrained=False
# ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd',
#                            model_math=precision, pretrained=False)

# if precision == 'fp16':
#     checkpoint_url = 'https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp16/versions/1/files/nvidia_ssdpyt_fp16_20190225.pt'
# elif precision == 'fp32':
#     checkpoint_url = 'https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/1/files/nvidia_ssdpyt_fp32_20190225.pt'

# checkpoint = torch.hub.load_state_dict_from_url('https://api.ngc.nvidia.com/v2/models/nvidia/ssdpyt_fp32/versions/1/files/nvidia_ssdpyt_fp32_20190225.pt', map_location="cpu")
# state_dict = {key.replace("module.", ""): value for key, value in checkpoint["state_dict"].items()}
# ssd_model.load_state_dict(state_dict)

In [14]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [7]:
# Set device type
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 5 disease classes
num_classes = 7

# Load pre-trained model with new head/tail
model = get_instance_segmentation_model(num_classes)
model.to(device)

# Build optimizer
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.AdamW(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# Learning rate scheduler (LR decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


In [ ]:
from engine import train_one_epoch, evaluate

# Training Loop
num_epochs = 10

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)